<a href="https://colab.research.google.com/github/jkurian49/Brushstroke-Classifier/blob/main/Brushstroke_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/jkurian49/Brushstroke-Classifier.git
%cd Brushstroke-Classifier/

# Data Loading and Preprocessing 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

data = pd.read_csv('brushstroke_data.csv')
data.replace('', np.nan, inplace=True)
data.dropna(inplace=True)

# CHANGE IMAGE PATH TO WORK IN GITHUB
image_path = "/content/drive/MyDrive/LABELED BRUSH STROKES/"

# Preprocess training images
TARGET_SIZE = (224,224,3) 
train_image = []

for i in tqdm(range(data.shape[0])): 
    img = image.load_img(image_path + data['ID'][i] + '.jpg',target_size=TARGET_SIZE)
    img = image.img_to_array(img)
    img = preprocess_input(img)
    train_image.append(img)
X = np.array(train_image)


# Training Model


In [ ]:
# model configuration per variable
config = {
    'Paint Type' : {
        'num_classes' : '2',
        'num_epochs' : '15'
    },
    'Brush Type' : {
        'num_classes' : '8',
        'num_epochs' : '20'
    },
    'Brush Material' : {
        'num_classes' : '2',
        'num_epochs' : '15'
    },
    'Tool Path Type' : {
        'num_classes' : '5',
        'num_epochs' : '20'
    }
}

In [ ]:
val_accs = {}
trained_models = {}
for config_name in config.keys():

  # specify single variable to be used as label 
  y = np.array(data[config_name])

  X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=420, test_size=0.2)

  # PARAMETERS
  NUM_CLASSES = config[config_name]['num_classes']
  BATCH_SIZE = 16

  from keras.applications.vgg16 import VGG16
  from keras.models import Model
  from keras.layers import Dense
  from keras.layers import Flatten
  from tensorflow import keras
  # load model without classifier layers
  model = VGG16(include_top=False, input_shape=(224, 224, 3))
  # mark loaded layers as not trainable
  for layer in model.layers:
    layer.trainable = False
  # add new classifier layers
  flat1 = Flatten()(model.layers[-1].output)
  class1 = Dense(1024, activation='relu')(flat1)
  output = Dense(NUM_CLASSES, activation='softmax')(class1)
  # define new model
  model = Model(inputs=model.inputs, outputs=output)
  # summarize
  # model.summary()

  model.compile(
      optimizer=keras.optimizers.RMSprop(),  # Optimizer
      # Loss function to minimize
      loss=keras.losses.SparseCategoricalCrossentropy(),
      # List of metrics to monitor
      metrics=[keras.metrics.SparseCategoricalAccuracy()],
  )
  history = model.fit(
      X_train,
      y_train,
      batch_size=BATCH_SIZE,
      epochs=int(config[config_name]['num_epochs']),
      # We pass some validation for
      # monitoring validation loss and metrics
      # at the end of each epoch
      validation_data=(X_valid, y_valid),
      verbose=2
  )
  final_val_acc = history.history['val_sparse_categorical_accuracy'][-1]
  val_accs[config_name]=final_val_acc

  # save model for future testing
  model.save(config_name +"_model")

In [ ]:
print(val_accs)

# Make Predictions

## Load Test Images

In [ ]:
import os

image_dir = "/content/Brushstroke-Classifier/test-images"

# Preprocess images
TARGET_SIZE = (224,224,3)
test_images = {}
for filename in os.listdir(image_dir):
    img = image.load_img(image_dir+"/"+filename,target_size=TARGET_SIZE)
    img = image.img_to_array(img)
    img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
    img = preprocess_input(img)
    test_images[filename] = img

## Test Model

In [ ]:
from collections import defaultdict

predictions = defaultdict(dict)
# first load each model
for config_name in config:
  model = keras.models.load_model(config_name +"_model")
  # give prediction for each test_image
  for filename,img in test_images.items():
    yhat = model.predict(img)
    label = np.argmax(yhat)
    predictions[filename][config_name] = label

# print results
for filename,label_dict in predictions.items():
  print(filename +":")
  for var_name,label in label_dict.items():
    print("{}:{}".format(var_name,label))